## Importar dataframe base

In [1]:
import pandas as pd
import numpy as np
# skempi db, a partir da qual será gerada a nossa
skempi = pd.read_csv("skempi_v2.csv", delimiter=";")
# df, nossa df já filtrada
df = pd.DataFrame()

## Filtrar apenas entradas com mutação para alanina

In [2]:
skempi_ala = skempi[skempi['Mutation(s)_PDB'].str.endswith('A')].reset_index(drop=True)

In [3]:
# Remover indicação de afinidade maior ou menor que X

# TODO não sei se devo fazer isso ou remover os dados
skempi_ala['Affinity_mut (M)'] = skempi_ala['Affinity_mut (M)'].str.replace('[<>]', '', regex=True)
skempi_ala['Affinity_wt (M)'] = skempi_ala['Affinity_wt (M)'].str.replace('[<>]', '', regex=True)

## Solução removendo-os
# skempi_ala['Affinity_mut (M)'] = skempi_ala.loc[~skempi_ala['Affinity_mut (M)'].str.startswith(('<', '>')), 'Affinity_mut (M)']
# skempi_ala['Affinity_wt (M)'] = skempi_ala.loc[~skempi_ala['Affinity_wt (M)'].str.startswith(('<', '>')), 'Affinity_wt (M)']


## Selecinar campos desejados

### pdb_id, partner1, partner2

In [4]:
df[['pdb_id', 'partner1', 'partner2']] = skempi_ala['#Pdb'].str.extract('(\w+)_(\w+)_(\w+)')

### partner_mut, resnum

In [5]:
df['partner_mut']= skempi_ala['Mutation(s)_PDB'].str[1]

In [6]:
df['resnum'] = skempi_ala['Mutation(s)_PDB'].str.extract('(\d+)')

### ΔG_wt, ΔG_mut, ΔΔG,  hotspot

Segundo o site da [documentação](https://life.bsc.es/pid/skempi2/info/faq_and_help#5) do banco de dados
>The affinities (Kd) of the wild-type complexes are in the column 'affinity_wt' and the affinities of the mutant are in the column 'affinity_mut'. These can be converted to ΔG values by the relationship **ΔG = RT ln Kd**; at room temperature this is **ΔG = (8.314/4184)*(273.15 + 25.0) * ln(wt)**, where ln() is the natural logarithm. The changes in affinity upon mutation is calculated as **ΔΔG = ΔGmut-ΔGwt**.
>

In [7]:
# Definir funções ΔG e ΔΔG
# Talvez seja necessário alterar valores de temperatura

calculate_dG = lambda kd: ((8.314/4184)*(273.15 + 25.0) * np.log(kd))
calculate_ddG =lambda dG_mut, dG_wt: dG_mut - dG_wt

In [8]:
# converter coluna para float
affinity_wt_column = skempi_ala.loc[(skempi_ala['Affinity_wt (M)'] != 'n.b'), 'Affinity_wt (M)'].astype(float)
affinity_mut_column = skempi_ala.loc[(skempi_ala['Affinity_mut (M)'] != 'n.b'), 'Affinity_mut (M)'].astype(float)

df['dG_mut'] = calculate_dG(affinity_mut_column)
df['dG_wt'] = calculate_dG(affinity_wt_column)

In [9]:
df['ddG'] = calculate_ddG(df['dG_mut'], df['dG_wt'])

In [10]:
df['hotspot'] = (df['ddG'] > 2.5).astype(int)

In [11]:
df.dropna().to_csv('alalin_filtered_skempi.csv', sep=';', index=False)